# Codeup Curriculum Anomaly Detection 

__Team:__ Daniel Ford, Joshua Mayes, Jared Angel

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from env import host, user, password, get_db_url

import acquire as aq
import prepare as pp
import warnings
warnings.filterwarnings('ignore')
import datetime

# Acquire and Clean

In [2]:
df = aq.wrangle_data()
df.head(3)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson
datetime,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,Hampton,#hampton,2015-09-22 00:00:00,2016-02-06 00:00:00,1,Full Stack PHP,None,None
2018-01-26 09:56:02,java-ii,1,8.0,Hampton,#hampton,2015-09-22 00:00:00,2016-02-06 00:00:00,1,Full Stack PHP,java-ii,None
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,Hampton,#hampton,2015-09-22 00:00:00,2016-02-06 00:00:00,1,Full Stack PHP,java-ii,object-oriented-programming


### This Data was collected From the curriculum logs database on the codeup server

### this data contains: 847,329 observations under 11 categories

#### for cleaning we:
   - Converted select data types
   - Changed Index to Datetime
   - Streamlined select names
   - drrop unnecessary or duplicate columns
   



### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)? and 7. Which lessons are least accessed?

Starting with data science program first

In [ ]:
# Eliminating the index homepage from lessons
trimmed = df[df.endpoint != '/']

In [ ]:
#Making a dataframe for just data science students
datascience = trimmed.loc[trimmed.program_id == 3]

In [ ]:
# Making dataframe grouped by cohort and lessons. Setting to descending order
ds_df = datascience.groupby('cohort_name').lesson.value_counts()
ds_df = ds_df.rename('total').reset_index()

In [ ]:
#Creating a for loop to run through each cohort and get most popular and least popular lessons, adding to dictionary
ds_cohorts = ds_df.cohort_name.unique()
most_common_ds = {}
lest_com_ds = {}
for cohort in ds_cohorts:
    pop_lesson_ds = ds_df.loc[ds_df.cohort_name == cohort].lesson.iloc[0]
    no_lesson_ds = ds_df.loc[ds_df.cohort_name == cohort].lesson.iloc[-1]
    most_common_ds.update({cohort: pop_lesson_ds})
    lest_com_ds.update({cohort : no_lesson_ds})

In [ ]:
# Making the dataframe from dictionary and renaming cols
most_com_ds = pd.DataFrame.from_dict(most_common_ds, orient='index')
most_com_ds.head()
most_com_ds.rename(columns = {'':'Cohort', 0:'Most Popular'}, inplace=True)

In [ ]:
# Making dataframe for least popular lessons
lest_com_ds = pd.DataFrame.from_dict(lest_com_ds, orient='index')
lest_com_ds.head()
lest_com_ds.rename(columns = {'':'Cohort', 0:'less Popular'}, inplace=True)

In [ ]:
# concatenating most and least along rows
ds_most_least = pd.concat([most_com_ds, lest_com_ds], axis=1)

In [ ]:
ds_most_least

Take aways:
-    The module 1 overview lesson accounted for 80% of the top lesson per cohort.
-    The least popular lesson was different for each cohort. However, opensecrets.org seems to be a counted as a lesson and could be wronly labeled

Full Stack PHP Program analysis

In [ ]:
# Making df for just php program
PHPstack = trimmed.loc[trimmed.program_id == 1]

In [ ]:
# Re arranging the data by cohort and lessons
PHP_df = PHPstack.groupby('cohort_name').lesson.value_counts()
PHP_df = PHP_df.rename('total').reset_index()

In [ ]:
#Creating a for loop to run through each cohort and get most popular and least popular lessons, adding to dictionary
cohorts = PHP_df.cohort_name.unique()
lest_comm_php = {}
most_common_php = {}
for cohort in cohorts:
    lesson = PHP_df.loc[PHP_df.cohort_name == cohort].lesson.iloc[0]
    most_common_php.update({cohort: lesson})
    less_lesson_php = PHP_df.loc[PHP_df.cohort_name == cohort].lesson.iloc[-1]
    lest_comm_php.update({cohort : less_lesson_php})

In [ ]:
#making most popular df
most_com_PHP = pd.DataFrame.from_dict(most_common_php, orient='index')
most_com_PHP.head()
most_com_PHP.rename(columns = {'':'Cohort', 0:'Most Popular'}, inplace=True)

In [ ]:
#Making least popular df
lest_com_PHP = pd.DataFrame.from_dict(lest_comm_php, orient='index')

lest_com_PHP.rename(columns = {'':'Cohort', 0:'Least Popular'}, inplace=True)

In [ ]:
# concatenating df1 and df2 along rows
php_most_least = pd.concat([most_com_PHP, lest_com_PHP], axis=1)

In [ ]:
php_most_least

Take aways:
- The most common overall lesson for the full stack java program is the fundamentals lesson.
- The least most common overall are work-in-teams and syntax-types-and-variables

Full stack java program

In [ ]:
# df just for java program
JAVAstack = trimmed.loc[trimmed.program_id == 2]

In [ ]:
#Rearrange the java df
java_df = JAVAstack.groupby('cohort_name').lesson.value_counts()

In [ ]:
# Reset index
java_df = java_df.rename('total').reset_index()

In [ ]:
# making list of cohort names
j_cohorts = java_df.cohort_name.unique()

In [ ]:
#Creating a for loop to run through each cohort and get most popular and least popular lessons, adding to dictionary
lest_com_j = {}
most_common_j = {}
for cohort in j_cohorts:
    lesson_j = java_df.loc[java_df.cohort_name == cohort].lesson.iloc[0]
    lest_lesson_j = java_df.loc[java_df.cohort_name == cohort].lesson.iloc[-1]
    most_common_j.update({cohort: lesson_j})
    lest_com_j.update({cohort : lest_lesson_j})

In [ ]:
# Making most common df
most_common_java = pd.DataFrame.from_dict(most_common_j, orient='index')
most_common_java.rename(columns = {'':'Cohort', 0:'Most Popular'}, inplace=True)

In [ ]:
#making least common df
lest_com_java = pd.DataFrame.from_dict(lest_com_j, orient='index')
lest_com_java.rename(columns = {'':'Cohort', 0:'Least Popular'}, inplace=True)

In [ ]:
# concatenating df1 and df2 along rows
java_most_least = pd.concat([most_common_java, lest_com_java], axis=1)

In [ ]:
java_most_least

Take aways:
-   The most common lessons for the full stack java program are fundamentals and introduction
-   The lest most common lessones are whack-a-mole and workbook

Frontend program

In [ ]:
#Making front end df
Frontend = trimmed.loc[trimmed.program_id == 4]

In [ ]:
# sorting by cohort and lesson
fe_df = Frontend.groupby('cohort_name').lesson.value_counts()

In [ ]:
fe_df

Take away:
-   The data is only available for one cohort and the data only shows the html-css lesson
-   Could not determine most and least from current data

### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

### 3. Are there students who, when active, hardly access the curriculum? 

### If so, what information do you have about these students?

In [4]:
students = pp.filter_students(df)

In [6]:
students.head(1)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson,date_mask
datetime,,,,,,,,,,,,
2019-11-04 16:54:38,html-css/introduction,539.0,52.0,Europa,#europa,2019-11-04 00:00:00,2020-04-17 00:00:00,2.0,Full Stack Java,html-css,introduction,True


# Takeaways:
   
   - The User IDs of students that accessed the curriculum the least are
       - 679.0
       - 812.0
       - 941.0
       - 956.0
       - 539.0
       - 572.0
   
   
   - 6 students respectively accessed the data less than 10 times while enrolled at codeup
       - of those 6 students: - 
           - 1 was enrolled in Data Science
           - 5 were enrolled in Full Stack Java
       - the Data Science student accessed the curriculum 10 times
       - the FS JAVA accessed the curriculum 20 times, with a median of 3
       
       
   - the least accessed modules were: 
       - Data Science; advanced topics 
       - FS JAVA; appendix

### 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? 

### Does it appear that any web-scraping is happening? 

### Are there any suspicious IP addresses?

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. 

### Do you see any evidence of that happening? Did it happen before?

### 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [8]:
java, php, data_sci, f_end = pp.filter_grads(df)

In [9]:
java.head(2)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson,mask
datetime,,,,,,,,,,,,
2018-01-26 11:36:38,javascript-i/functions,26.0,18.0,Pinnacles,#pinnacles,2017-03-27 00:00:00,2017-07-20 00:00:00,2.0,Full Stack Java,javascript-i,functions,True
2018-01-26 13:57:51,spring/extra-features/file-upload,40.0,16.0,Niagara,#niagara,2016-10-26 00:00:00,2017-03-09 00:00:00,2.0,Full Stack Java,spring,extra-features,True


In [10]:
php.head(2)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson,mask
datetime,,,,,,,,,,,,
2018-01-26 09:56:05,java-ii/object-oriented-programming,1.0,8.0,Hampton,#hampton,2015-09-22 00:00:00,2016-02-06 00:00:00,1.0,Full Stack PHP,java-ii,object-oriented-programming,True
2018-01-26 09:56:06,slides/object_oriented_programming,1.0,8.0,Hampton,#hampton,2015-09-22 00:00:00,2016-02-06 00:00:00,1.0,Full Stack PHP,slides,object_oriented_programming,True


In [11]:
data_sci.head(2)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson,mask
datetime,,,,,,,,,,,,
2020-01-30 08:01:31,1-fundamentals/1.1-intro-to-data-science,485.0,34.0,Bayes,#,2019-08-19 00:00:00,2020-01-30 00:00:00,3.0,Data Science,1-fundamentals,1.1-intro-to-data-science,True
2020-01-30 08:01:32,1-fundamentals/modern-data-scientist.jpg,485.0,34.0,Bayes,#,2019-08-19 00:00:00,2020-01-30 00:00:00,3.0,Data Science,1-fundamentals,modern-data-scientist.jpg,True


In [12]:
f_end.head(2)

,endpoint,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program,module,lesson,mask
datetime,,,,,,,,,,,,
2018-03-22 19:01:54,content/html-css,152.0,9.0,Apollo,#apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,4.0,Front-End,content,html-css,True
2018-03-22 19:01:54,content/html-css/gitbook/images/favicon.ico,152.0,9.0,Apollo,#apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,4.0,Front-End,content,html-css,True


### There are 568 unique graduates by ID
#### of those 568, 48 are staff
#### staff was removed as it may bias the data


| **Programs**    	| **Top Module** 	| **Amount Accessed** 	| **Top Lessons**       	| **Amount accessed** 	|
|:-----------------	|:----------------	|:---------------------:	|:-----------------------	|:---------------------:	|
| Full Stack JAVA 	| Spring         	|         7270        	| Fundamentals          	|         5400        	|
|                 	| Javascrip      	|         6725        	| Introduction          	|         2981        	|
|                 	| HTML-CSS       	|         4850        	| Search_index.json     	|         2370        	|
| Full Stack PHP  	| Content        	|         6397        	| Fundamentals          	|         1442        	|
|                 	| Javascript-I   	|         2972        	| Laravel               	|         1188        	|
|                 	| HTML-CSS       	|         1921        	| Introduction          	|         1156        	|
| Data Science    	| Fundementals   	|         1053        	| Overview              	|         674         	|
|                 	| SQL            	|         1046        	| Search_index.json     	|         493         	|
|                 	| Classification 	|         1036        	| AI-ML-DL-timeline.jpg 	|         316         	|
| Front End       	| Content        	|          4          	| HTML-CSS              	|          4          	|